In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import scipy.io as io
from settings_config import *
from dimensionality_reduction import *
from preprocessing import *
from sklearn import svm, naive_bayes,discriminant_analysis,model_selection
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.decomposition import PCA

## Get data

In [2]:
# Get data from csv
likelihood_EMT46_2, positions_EMT46_2, angles_EMT46_2 = get_likelihood_positions_angles_nans(EMT46_2_path,length_EMT46_2,\
                                                                              likelihood_threshold=0.75,labels_format='46/16')
likelihood_EMT16, positions_EMT16, angles_EMT16 = get_likelihood_positions_angles_nans(EMT16_path,length_EMT16,\
                                                                              likelihood_threshold=0.8,labels_format='46/16')

likelihood_EMT19, positions_EMT19, angles_EMT19 = get_likelihood_positions_angles_nans(EMT19_path,length_EMT19,\
                                                                                              likelihood_threshold=0.75)
likelihood_EMT28, positions_EMT28, angles_EMT28 = get_likelihood_positions_angles_nans(EMT28_path,length_EMT28,\
                                                                                              likelihood_threshold=0.75)

likelihood_EMT48, positions_EMT48, angles_EMT48 = get_likelihood_positions_angles_nans(EMT48_path,length_EMT48,\
                                                                                              likelihood_threshold=0.75)
likelihood_EMT17, positions_EMT17, angles_EMT17 = get_likelihood_positions_angles_nans(EMT17_path,length_EMT17,\
                                                                                              likelihood_threshold=0.75)
likelihood_EMT47, positions_EMT47, angles_EMT47 = get_likelihood_positions_angles_nans(EMT47_path,length_EMT47,\
                                                                                              likelihood_threshold=0.75)
likelihood_EMT18, positions_EMT18, angles_EMT18 = get_likelihood_positions_angles_nans(EMT18_path,length_EMT18,\
                                                                                              likelihood_threshold=0.75)

Deleted ouliers: 2
Deleted ouliers: 0
Deleted ouliers: 4
Deleted ouliers: 0
Deleted ouliers: 0
Deleted ouliers: 0
Deleted ouliers: 37
Deleted ouliers: 4
Deleted ouliers: 50
Deleted ouliers: 4
Deleted ouliers: 1
Deleted ouliers: 1
Deleted ouliers: 17
Deleted ouliers: 10
Deleted ouliers: 0
Deleted ouliers: 0
Deleted ouliers: 0
Deleted ouliers: 0
Deleted ouliers: 9
Deleted ouliers: 8
Deleted ouliers: 9
Deleted ouliers: 4
Deleted ouliers: 26
Deleted ouliers: 20
Deleted ouliers: 8
Deleted ouliers: 6
Deleted ouliers: 7
Deleted ouliers: 0
Deleted ouliers: 11
Deleted ouliers: 1
Deleted ouliers: 2
Deleted ouliers: 0
Deleted ouliers: 10
Deleted ouliers: 2
Deleted ouliers: 19
Deleted ouliers: 7
Deleted ouliers: 33
Deleted ouliers: 0
Deleted ouliers: 20
Deleted ouliers: 25
Deleted ouliers: 5
Deleted ouliers: 0
Deleted ouliers: 0
Deleted ouliers: 0
Deleted ouliers: 17
Deleted ouliers: 22
Deleted ouliers: 23
Deleted ouliers: 38
Deleted ouliers: 53
Deleted ouliers: 13
Deleted ouliers: 73
Deleted ouli

In [4]:
angular_velocity_EMT16 = delete_nans(abs(np.diff(angles_EMT16)))
angular_velocity_EMT46_2 = delete_nans(abs(np.diff(angles_EMT46_2)))
angular_velocity_EMT19 = delete_nans(abs(np.diff(angles_EMT19)))
angular_velocity_EMT28 = delete_nans(abs(np.diff(angles_EMT28)))
angular_velocity_EMT17 = delete_nans(abs(np.diff(angles_EMT17)))
angular_velocity_EMT18 = delete_nans(abs(np.diff(angles_EMT18)))
angular_velocity_EMT47 = delete_nans(abs(np.diff(angles_EMT47)))
angular_velocity_EMT48 = delete_nans(abs(np.diff(angles_EMT48)))

# Delete nans
angles_EMT16 = delete_nans(angles_EMT16)
angles_EMT46_2 = delete_nans(angles_EMT46_2)
angles_EMT19 = delete_nans(angles_EMT19)
angles_EMT28 = delete_nans(angles_EMT28)
angles_EMT17 = delete_nans(angles_EMT17)
angles_EMT48 = delete_nans(angles_EMT48)
angles_EMT18 = delete_nans(angles_EMT18)
angles_EMT47 = delete_nans(angles_EMT47)

angles_EMT16_norm = normalize_angles_range(angles_EMT16)
angles_EMT46_2_norm = normalize_angles_range(angles_EMT46_2)
angles_EMT19_norm = normalize_angles_range(angles_EMT19)
angles_EMT28_norm = normalize_angles_range(angles_EMT28)
angles_EMT17_norm = normalize_angles_range(angles_EMT17)
angles_EMT48_norm = normalize_angles_range(angles_EMT48)
angles_EMT18_norm = normalize_angles_range(angles_EMT18)
angles_EMT47_norm = normalize_angles_range(angles_EMT47)

There are 3930 frames with high likelihood
There are 4871 frames with high likelihood
There are 10954 frames with high likelihood
There are 10979 frames with high likelihood
There are 11621 frames with high likelihood
There are 12457 frames with high likelihood
There are 14314 frames with high likelihood
There are 13194 frames with high likelihood
There are 4207 frames with high likelihood
There are 5927 frames with high likelihood
There are 11485 frames with high likelihood
There are 11451 frames with high likelihood
There are 11993 frames with high likelihood
There are 13661 frames with high likelihood
There are 12868 frames with high likelihood
There are 14907 frames with high likelihood
max_trunk: 1.968851095081795
min_trunk: 1.1347295154690478
max_head: 2.0226253318108562
min_head: 0.40070368416317986
max_trunk: 1.9831175648711548
min_trunk: 1.1784662246097244
max_head: 1.7500260353685604
min_head: 0.5210298071265259
max_trunk: 2.0044901162011812
min_trunk: 1.1012426312904018
max_

In [154]:
labels_names = ['Early mild 1','Early mild 2','Early mild 3','Early mild 4','Late mild 1','Late mild 2','Late mild 3','Late mild 4']

angles =  [angles_EMT19,angles_EMT16,angles_EMT17,angles_EMT18,angles_EMT28,angles_EMT46_2,angles_EMT48,angles_EMT47]
angular_velocity = [angular_velocity_EMT19,angular_velocity_EMT16,angular_velocity_EMT17,angular_velocity_EMT18,angular_velocity_EMT28,angular_velocity_EMT46_2,angular_velocity_EMT48,angular_velocity_EMT47]
angles_norm =  [angles_EMT19_norm,angles_EMT16_norm,angles_EMT17_norm,angles_EMT18_norm,angles_EMT28_norm,angles_EMT46_2_norm,angles_EMT48_norm,angles_EMT47_norm]


In [155]:
def get_data_classifier_format(videos, babies=[0,0,0,0,1,1,1,1]):
    y = []
    for i,label in enumerate(babies):
        if label == 0:
            y.append(np.zeros(videos[i].shape[1]))
        else:
            y.append(np.ones(videos[i].shape[1]))
    y = np.concatenate(y, axis=0)
    X = np.concatenate(videos,axis=1)
    return X.T,y

def evaluate_shuffled_labels(classifier,X,y=[0,0,0,0,1,1,1,1]):
    X,y = get_data_classifier_format(X,y)
    accs = []
    for i in range(10):
        y_shuf = y.copy()
        random.shuffle(y_shuf)
        accs.append(model_selection.cross_val_score(classifier, X, y_shuf, cv=10))
    acc = np.mean(np.array(accs))
    print('Accuracy by chance: {}%'.format(np.round(acc*100)))
    return acc

def evaluate_lda_svm(X,y=[0,0,0,0,1,1,1,1]):
    X,y = get_data_classifier_format(X,y)
    lda = discriminant_analysis.LinearDiscriminantAnalysis()
    clf = svm.SVC(kernel='linear')
    svrs = [lda, clf]
    names = ['LDA','SVM linear']
    for svr,name in zip(svrs,names):
        print(' - {}'.format(name))
        accuracy = model_selection.cross_val_score(svr, X, y, cv=10)
        print('Accuracy: {} %'.format(np.round(np.mean(accuracy)*100)))
        acc_chance = evaluate_shuffled_labels(svr,X, y)
        print('')

def evaluate_classifier(classifier,X,y=[0,0,0,0,1,1,1,1]):
    accs = []
    for i,video in enumerate(X):
        videos_copy = X.copy()
        del videos_copy[i]
        X_angles,y_angles=get_data_classifier_format(videos_copy,np.delete(y,i))

        classifier.fit(X_angles,y_angles)
        y_pred = classifier.predict(X[i].T)

        if y[i]==0:
            y_true = np.zeros(X[i].shape[1])
        else:
            y_true = np.ones(X[i].shape[1])

        acc = accuracy_score(y_true,y_pred)
        accs.append(acc)
    print('Accuracy: {} %'.format(np.round(np.mean(accs)*100)))
    acc_chance = evaluate_shuffled_labels(classifier,X,y)
    print('')
        
def evaluate_classifier_common_manifold(classifier,n_components,X,Y=[0,0,0,0,1,1,1,1]):
    pca = PCA(n_components=n_components)
    pca.fit(np.concatenate(X,axis=1).T)

    X_projected = []
    for video in X:
        X_projected.append(pca.transform(video.T).T)
    evaluate_classifier(classifier,X_projected,Y)
    
def evaluate_classifier_individual_manifolds(classifier,n_components,X,Y=[0,0,0,0,1,1,1,1]):
    X_projected = []
    for video in X:
        pca = PCA(n_components=n_components)
        pca.fit(video.T)
        X_projected.append(pca.transform(video.T).T)
    print(X_projected)
    evaluate_classifier(classifier,X_projected,Y)

# LDA on angles, normalized angles and angular velocity

In [156]:
print('LDA')
print('')
print('Angles')
evaluate_classifier(discriminant_analysis.LinearDiscriminantAnalysis(),angles)
print('Angles normalized')
evaluate_classifier(discriminant_analysis.LinearDiscriminantAnalysis(),angles_norm)
print('Angular velocity')
evaluate_classifier(discriminant_analysis.LinearDiscriminantAnalysis(),angular_velocity)
print('')

LDA

Angles
Accuracy: 61.0 %
Accuracy by chance: 53.0%

Angles normalized
Accuracy: 89.0 %
Accuracy by chance: 53.0%

Angular velocity
Accuracy: 27.0 %
Accuracy by chance: 53.0%




# LDA on angles, normalized angles and angular velocity projected onto a common manifold

In [157]:
print('LDA')
print('')
print('Angles')
evaluate_classifier_common_manifold(discriminant_analysis.LinearDiscriminantAnalysis(),6,angles)
print('Angles normalized')
evaluate_classifier_common_manifold(discriminant_analysis.LinearDiscriminantAnalysis(),6,angles_norm)
print('Angular velocity')
evaluate_classifier_common_manifold(discriminant_analysis.LinearDiscriminantAnalysis(),6,angular_velocity)
print('')

LDA

Angles
Accuracy: 67.0 %
Accuracy by chance: 53.0%

Angles normalized
Accuracy: 66.0 %
Accuracy by chance: 53.0%

Angular velocity
Accuracy: 28.0 %
Accuracy by chance: 53.0%


